In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import torch

k:\GIT_PROJECT\vecoder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### CodeLlama 7B

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf", cache_dir="./models")
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf", cache_dir="./models")

In [48]:
from transformers import pipeline
import torch

code_continuator = pipeline('text-generation',
                            model=model,
                            tokenizer=tokenizer,
                            torch_dtype=torch.float16,
                            config={"temperature": 0.2},
                            device=0
                            )

In [51]:
def continue_code(prompt):
    sequences = code_continuator(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=64,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
        
def generate_code(prompt):
    prompt = "requirements: " + prompt + "/n code: "
    
    sequences = code_continuator(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=64,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [53]:
continue_code("gives a python function that generate prime numbers under 100: ")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [54]:
generate_code("gives a python function that generate prime numbers under 100")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: requirements: gives a python function that generate prime numbers under 100/n code: 

```python
def generate_prime_numbers(n):
    def is_prime(n):
        for i in range(2, n):
            if n % i == 0:


### GPT2

In [3]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", cache_dir="./models")
model = AutoModelForCausalLM.from_pretrained("gpt2", cache_dir="./models")

In [4]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [5]:
generator("def printPrime(", max_length=64, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'def printPrime(x);\n\nNow using the same command, you can do the same thing in Python, again with Python 3 support:\n\nimport sys from time import time import str from time import time2fdef printPrime(x)\n\nAs your Python interpreter is running, you can get what'}]

### Glaive Coder 7B

In [2]:
tokenizer = AutoTokenizer.from_pretrained("glaiveai/glaive-coder-7b", cache_dir="./models")
model = AutoModelForCausalLM.from_pretrained("glaiveai/glaive-coder-7b", cache_dir="./models")

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [ ]:
def fmt_prompt(prompt):
    return f"<s> [INST] {prompt} [/INST]"

inputs = tokenizer(fmt_prompt(prompt),return_tensors="pt").to(model.device)

outputs = model.generate(**inputs,do_sample=True,temperature=0.1,top_p=0.95,max_new_tokens=100)

print(tokenizer.decode(outputs[0],skip_special_tokens=True,clean_up_tokenization_spaces=False))